In [2]:
#Value at Risk,VaR, is a risk management technique to calculate the maximum loss an investment portfolio is likely to face in a specified time frame with a certain degree of confidence. 
#It is a commonly calculated metric used within a suite of financial metrics and models to help aid investment decisions.

# Using the Variance-Covariance Method,I have calculated VaR for a portfolio of equities,using historical stock prices from Yahoo!Finance

#The steps I undertook are as below


# Import the libraries:Panda (reading data, manipulation, analysis), numpy(for mathematical array operations)

import pandas as pd
import numpy as np


In [3]:
# To retrieve stock prices from Yahoo Finance, I decided to download more libraries
# I installed all the needed libraries, such as panda datareaders,datetime and Yahoo finance 
# Since they decommissioned their historical data API, Panda data reader stopped working with Yahoo Finance data sets
# Fix_yahoo_finance, the original library, which I had tried to install, was a mere fix.
# However, for reasons of backward competability, yfinance was developed to download historical market data from Yahoo! Finance.
# Hence, I pip-installed yfinance directly

from pandas_datareader import data as pdr
import yfinance as yf
import datetime as dt


In [4]:
# I created a sample portfolio of equities
tickers = ['AAPL', 'FB', 'C', 'DIS']

In [5]:
# I indicated an arbirary weight allocation to portfolio stocks
weights = np.array([.25, .3, .15, .3])

In [6]:
# I set the initial investment amount
initial_investment = 2500000

In [8]:
#Reading the closing prices prices from yahoo finance
data = pdr.get_data_yahoo(tickers, start='2018-01-01', end = dt.date.today())['Close']

In [9]:
# I decided to calculate the periodic returns per selected stocks
returns= data.pct_change()

In [11]:
returns.head()

Symbols,AAPL,FB,C,DIS
Date,,,,
2018-01-02,NaN,NaN,NaN,NaN
2018-01-03,-0.000174,0.017914,0.003093,0.004293
2018-01-04,0.004645,-0.001841,0.012334,-0.000445
2018-01-05,0.011385,0.013671,-0.001324,-0.005435
2018-01-08,-0.003714,0.007653,-0.011670,-0.014334


In [12]:
# To explore some key statistics:

returns.describe()

Symbols,AAPL,FB,C,DIS
count,765.000000,765.000000,765.000000,765.000000
mean,0.001661,0.000716,0.000193,0.000776
std,0.022112,0.023918,0.027637,0.020941
min,-0.128647,-0.189609,-0.192986,-0.129846
25%,-0.007926,-0.010119,-0.010436,-0.007841
50%,0.001570,0.001352,0.000271,0.000343
75%,0.012217,0.012624,0.011129,0.009174
max,0.119808,0.108164,0.179843,0.144123


In [15]:
# A underlying assumption of VaR model is that stock market returns follow a normal distribution. 
# Stock returns are assumed to be a normal continuous random variable
#I installed the Scipy.stat module to further explore the probability distribution of returns and other statistical functions
# For this purpose, Norm, an object- which is an instance of the rv_continuous class, is adopted

from scipy.stats import norm

#I decided to calculate the var-covariance matrix using the returns calculated earlier;

cov_matrix= returns.cov()

cov_matrix

Symbols,AAPL,FB,C,DIS
Symbols,,,,
AAPL,0.000489,0.000329,0.000313,0.000222
FB,0.000329,0.000572,0.000276,0.000205
C,0.000313,0.000276,0.000764,0.000373
DIS,0.000222,0.000205,0.000373,0.000439


In [19]:
# Calculating the mean returns for each stock

average_returns= returns.mean()

average_returns

Symbols
AAPL    0.001661
FB      0.000716
C       0.000193
DIS     0.000776
dtype: float64

In [22]:
#Overall portfolio average returns,using the dot product formular for the means against investment weights

Portf_mean =average_returns. dot(weights)

Portf_mean


0.0008915847695011031

In [36]:
# Portfolio Standard Deviation is calculated using :

Portf_stdev= round(np.sqrt(weights.T.dot(cov_matrix).dot(weights)),4)

Portf_stdev

0.0184

In [34]:
#Calculating the mean value of the given investment

Mean_investment = round((1+ Portf_mean)*initial_investment,4)

Mean_investment

2502228.9619

In [37]:
# Standard deviation of the investment

Stdev_investment = initial_investment* Portf_stdev

Stdev_investment

46000.0

In [38]:
# I then proceed to choose the confidence interval of interest. say 95  per cent

Conf_level1 = 0.05

In [40]:
# Using the Scipy percentage point function(ppf) method to generate values for the inverse cummulative density function 
#to a normal distribution, I plugged in the mean and stdev of the portfolio calculated earlier

#It takes a percentage and returns a standard deviation multiplier for what value that percentage occurs at
# For instance, norm.ppf(0.90, loc= 134, scale =3.45) will return a value(that functions as standard-deviation multiplier) marking
# where 95% of data point would be contained if our data follows a normal distribution. 

Cutoff1 =norm.ppf(Conf_level1, Mean_investment, Stdev_investment)

Cutoff1

2426565.695060232

In [44]:
# Eventually, I am able to calculate the VaR at the confidence interval chosen. 
# Interpretation: By this result, we are saying that the loss in our portfolio will not exceed $73,000 over 1-day period @95 C.I

VaR_1dl = round(initial_investment- Cutoff1,3)

VaR_1dl

73434.305

In [45]:
# What if we decided to evaluate this over a bigger window of time? 
# In this case,we take 1-day VaR multiplied by square root of the select time period
# That is, in order to calculate n-Day VaR
# Observation:  As the the time window is expanded as below, the potential loss tends to increase. This seems logical

num_days = 30

for x in range(1, num_days+1):
     
     print(str(x) + " day VaR @ 95% Confidence: " + str(np.round(VaR_1dl * np.sqrt(x),2)))

1 day VaR @ 95% Confidence: 73434.3
2 day VaR @ 95% Confidence: 103851.79
3 day VaR @ 95% Confidence: 127191.95
4 day VaR @ 95% Confidence: 146868.61
5 day VaR @ 95% Confidence: 164204.1
6 day VaR @ 95% Confidence: 179876.58
7 day VaR @ 95% Confidence: 194288.91
8 day VaR @ 95% Confidence: 207703.58
9 day VaR @ 95% Confidence: 220302.91
10 day VaR @ 95% Confidence: 232219.66
11 day VaR @ 95% Confidence: 243554.04
12 day VaR @ 95% Confidence: 254383.89
13 day VaR @ 95% Confidence: 264771.15
14 day VaR @ 95% Confidence: 274766.01
15 day VaR @ 95% Confidence: 284409.84
16 day VaR @ 95% Confidence: 293737.22
17 day VaR @ 95% Confidence: 302777.4
18 day VaR @ 95% Confidence: 311555.37
19 day VaR @ 95% Confidence: 320092.71
20 day VaR @ 95% Confidence: 328408.2
21 day VaR @ 95% Confidence: 336518.26
22 day VaR @ 95% Confidence: 344437.42
23 day VaR @ 95% Confidence: 352178.55
24 day VaR @ 95% Confidence: 359753.15
25 day VaR @ 95% Confidence: 367171.52
26 day VaR @ 95% Confidence: 374442.95
